# Configure KBase Jupyter Dev Environment

In [1]:
%run patricutil.py
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

python version 3.9.13
KBBaseModules 0.0.1


1716008752.888874 INFO: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
1716008752.889476 INFO: NumExpr defaulting to 8 threads.


modelseedpy 0.3.3
cobrakbase 0.3.1
Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


# Loading gene list from PATRIC

In [35]:
gene_ids = {}
with open('data/gene.lst', 'r') as file:
    content = file.read()
genes = content.split('\n')
for gene_id in genes:
    gene_ids[gene_id] = None
util.save("genes", gene_ids)

# Pulling sequence MD5s

In [36]:
genes = util.load("genes")
gene_list = []
bundle_size = 100
for i,gene_id in enumerate(genes.keys()):
    if genes[gene_id] is None:
        gene_list.append(gene_id)
    if len(gene_list) >= bundle_size:
        query = ",".join(gene_list)
        url = f'https://www.patricbrc.org/api/genome_feature/?in(patric_id,({query}))&select(patric_id,na_sequence_md5)&http_accept=application/json'
        response = requests.get(url)
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            for i,gene in enumerate(data):
                genes[data[i]['patric_id']] = data[i]['na_sequence_md5']
            util.save("genes",genes)
        gene_list = []
if len(gene_list) > 0:
    query = ",".join(gene_list)
    url = f'https://www.patricbrc.org/api/genome_feature/?in(patric_id,({query}))&select(na_sequence_md5)&http_accept=application/json'
    response = requests.get(url)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        for i,gene in enumerate(gene_list):
            genes[gene] = data[i]['na_sequence_md5']
        util.save("genes",genes)


# Pulling sequences based on MD5

In [ ]:
genes = util.load("genes")
gen_seqs = {}
gene_list = []
gene_ids = []
for i,gene_id in enumerate(genes.keys()):
    if genes[gene_id] is not None:
        gene_list.append(genes[gene_id])
        gene_ids.append(gene_id)
    if len(gene_list) >= 1000:
        query = ",".join(gene_list)
        url = f'https://www.patricbrc.org/api/feature_sequence/?in(md5,({gene_list}))&select(sequence)&http_accept=application/json'
        response = requests.get(url)
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            for i,gene in enumerate(gene_ids):
                gen_seqs[gene] = data[i]['sequence']
            util.save("genes",genes)
        gene_list = []
        gene_ids = []
if len(gene_list) > 0:
    query = ",".join(gene_list)
    url = f'https://www.patricbrc.org/api/feature_sequence/?in(md5,({gene_list}))&select(sequence)&http_accept=application/json'
    response = requests.get(url)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        for i,gene in enumerate(gene_ids):
            gen_seqs[gene] = data[i]['sequence']
        util.save("genes",genes)

# Querying single gene

In [ ]:
# Replace 'gene_id' with your PATRIC gene ID
gene_id = 'fig|1352.9102.rna.4'

# Define the URL for the PATRIC API
url = f'https://www.patricbrc.org/api/feature_sequence/?eq(gene_id,{gene_id})&select(sequence)&http_accept=application/json'

# Send a GET request to the PATRIC API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Get the gene sequence
    gene_sequence = data[0]['sequence']

    print(gene_sequence)
else:
    print('Failed to get gene sequence')

Failed to get gene sequence
